# Notebook 3 — Comprendre les algorithmes d'apprentissage automatiques + SVM & clustering hiérarchique

Ce notebook est en **deux parties** :
1. **Construire k-means étape par étape** (en « briques ») pour comprendre ce qui se passe :
   - choix de la **loss** (fonction objectif)
   - assignation aux centres
   - mise à jour des centroïdes
   - critère d'arrêt
2. **SVM + clustering hiérarchique** : réglage des hyperparamètres, métriques, accuracy, matrice de confusion.

 **Datasets utilisés (accessibles sans téléchargement)** :
- `make_blobs` (données synthétiques générées par scikit-learn) pour le clustering
- `load_digits` (dataset intégré à scikit-learn) pour la classification SVM

> Certaines cellules sont à compléter : cherchez `TODO`.


In [ ]:
# --- Imports & config ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs, load_digits
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    ConfusionMatrixDisplay
)

np.random.seed(42)


---
## Partie 1 — k-means « from scratch »

### 1) Génération d'un dataset de clustering
On génère des points en 2D avec `make_blobs` pour visualiser facilement.


In [ ]:
# Dataset synthétique (clustering)
X, y_true = make_blobs(
    n_samples=400,
    centers=4,
    cluster_std=1.2,
    random_state=42
)

plt.figure()
plt.scatter(X[:, 0], X[:, 1], s=15)
plt.title("Données (make_blobs) — pas de labels pour k-means")
plt.xlabel("x1")
plt.ylabel("x2")
plt.show()

print("X shape:", X.shape)


### 2) Définir la loss (fonction objectif) de k-means
La version classique de k-means minimise la **somme des distances au carré** aux centroïdes :
\[
J = \sum_{i=1}^{n} \|x_i - \mu_{c(i)}\|^2
\]
où $c(i)$ est le cluster assigné au point $x_i$.

**À faire :** compléter la fonction `kmeans_loss`.
- Entrées : `X` (n×d), `centroids` (k×d), `labels` (n)
- Sortie : un scalaire (loss)


In [ ]:
def kmeans_loss(X, centroids, labels):
    """Somme des distances au carré aux centroïdes (inertia).
    TODO: implémenter la loss.
    """
    # TODO:
    # 1) pour chaque point i, récupérer son centroïde centroids[labels[i]]
    # 2) calculer ||x_i - mu||^2 et sommer
    loss = ...  # TODO
    return loss

# --- mini test (doit tourner après votre implémentation) ---
# centroids_test = X[:4]
# labels_test = np.zeros(X.shape[0], dtype=int)
# print(kmeans_loss(X, centroids_test, labels_test))


### 3) Étape A — Initialisation des centroïdes
Plusieurs stratégies :
- choix aléatoire de points (simple)
- k-means++ (meilleur, mais plus avancé)

**À faire :** initialiser `k` centroïdes en sélectionnant `k` points au hasard dans `X`.


In [ ]:
def init_centroids_random(X, k, random_state=42):
    rng = np.random.default_rng(random_state)
    # TODO: tirer k indices sans remise (replace=False)
    idx = ...  # TODO
    return X[idx]

k = 4
centroids = init_centroids_random(X, k)

plt.figure()
plt.scatter(X[:, 0], X[:, 1], s=15)
plt.scatter(centroids[:, 0], centroids[:, 1], s=120, marker="X")
plt.title("Initialisation aléatoire des centroïdes")
plt.show()


### 4) Étape B — Assignation (E-step)
On assigne chaque point au centroïde **le plus proche**.

**À faire :** compléter `assign_labels`.


In [ ]:
def assign_labels(X, centroids):
    """Retourne labels (n,) : l'indice du centroïde le plus proche pour chaque point."""
    # TODO:
    # 1) calculer les distances au carré entre chaque point et chaque centroïde
    #    astuce: broadcasting -> (n,1,d) - (1,k,d) -> (n,k,d)
    # 2) argmin sur l'axe des clusters
    labels = ...  # TODO
    return labels

# labels = assign_labels(X, centroids)
# print(np.bincount(labels))


### 5) Étape C — Mise à jour des centroïdes (M-step)
Chaque centroïde devient la **moyenne** des points qui lui sont assignés.

**À faire :** compléter `update_centroids`.
- Attention au cas où un cluster se vide (aucun point assigné) : stratégie simple = réinitialiser au hasard.


In [ ]:
def update_centroids(X, labels, k, random_state=42):
    rng = np.random.default_rng(random_state)
    d = X.shape[1]
    new_centroids = np.zeros((k, d))

    for j in range(k):
        cluster_points = X[labels == j]
        if len(cluster_points) == 0:
            # TODO: cluster vide -> réinitialiser avec un point aléatoire
            new_centroids[j] = ...  # TODO
        else:
            # TODO: moyenne des points
            new_centroids[j] = ...  # TODO

    return new_centroids


### 6) Boucle k-means complète + critère d'arrêt
On répète : assignation → mise à jour, jusqu'à convergence.

**Critères d'arrêt possibles :**
- nombre max d'itérations
- déplacement des centroïdes < tolérance
- amélioration de la loss < tolérance

**À faire :** compléter `kmeans_fit`.


In [ ]:
def kmeans_fit(X, k, max_iter=50, tol=1e-4, random_state=42):
    centroids = init_centroids_random(X, k, random_state=random_state)
    prev_loss = None
    history = []

    for it in range(max_iter):
        # TODO: assigner labels
        labels = ...  # TODO

        # TODO: calculer la loss
        loss = ...  # TODO
        history.append(loss)

        # TODO: mettre à jour centroids
        new_centroids = ...  # TODO

        # critère d'arrêt (loss)
        if prev_loss is not None and abs(prev_loss - loss) < tol:
            print(f"Convergence: it={it}, loss={loss:.3f}")
            centroids = new_centroids
            break

        centroids = new_centroids
        prev_loss = loss

    return centroids, labels, history

# --- Exécution protégée ---
try:
    centroids_f, labels_f, history = kmeans_fit(X, k=4, max_iter=80, tol=1e-3, random_state=42)

    plt.figure()
    plt.plot(history, marker="o")
    plt.xlabel("itération")
    plt.ylabel("loss (inertia)")
    plt.title("Évolution de la loss de k-means")
    plt.show()

    plt.figure()
    plt.scatter(X[:, 0], X[:, 1], c=labels_f, s=15)
    plt.scatter(centroids_f[:, 0], centroids_f[:, 1], s=140, marker="X")
    plt.title("Résultat k-means (from scratch)")
    plt.show()
except Exception as e:
    print("⚠️ Complétez les TODO de la Partie 1 pour exécuter k-means:", e)


### 7) Comparaison rapide avec `sklearn.cluster.KMeans`
Quand votre implémentation marche, comparez le résultat (qualitativement) avec `sklearn`.


In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=4, n_init=20, random_state=42)
labels_sk = km.fit_predict(X)

plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=labels_sk, s=15)
plt.scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1], s=140, marker="X")
plt.title("k-means (scikit-learn)")
plt.show()


---
## Partie 2 — SVM, clustering hiérarchique, hyperparamètres et métriques

### 8) Dataset de classification : `load_digits`
On charge des images 8×8 de chiffres (0..9). C’est un dataset classique et intégré à scikit-learn.


In [ ]:
digits = load_digits()
X = digits.data          # (n_samples, 64)
y = digits.target        # (n_samples,)
print("X:", X.shape, " y:", y.shape)
print("Classes:", np.unique(y))

# Visualiser quelques exemples
fig, axes = plt.subplots(2, 6, figsize=(10, 3))
for ax, idx in zip(axes.ravel(), range(12)):
    ax.imshow(digits.images[idx], cmap="gray")
    ax.set_title(str(y[idx]))
    ax.axis("off")
plt.tight_layout()
plt.show()


### 9) Split train/test + baseline SVM
SVM est sensible à l'échelle : on utilise un `StandardScaler`.

**À faire :**
- compléter les paramètres de `train_test_split`
- entraîner un SVM RBF (`SVC`) et calculer accuracy + matrice de confusion


In [ ]:
from sklearn.svm import SVC

# TODO: split reproductible (test_size 0.2, random_state 42, stratify y)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=...,      # TODO
    random_state=...,   # TODO
    stratify=...        # TODO
)

svm_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(
        kernel="rbf",
        C=...,          # TODO: ex 10
        gamma=...       # TODO: ex "scale" ou 0.01
    ))
])

try:
    svm_pipe.fit(X_train, y_train)
    y_pred = svm_pipe.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    print("Accuracy test:", acc)

    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title("Matrice de confusion — SVM baseline")
    plt.show()

    print("\nRapport de classification:\n", classification_report(y_test, y_pred))
except Exception as e:
    print(" Complétez les TODO SVM (C, gamma, split) pour exécuter:", e)


### 10) Réglage des hyperparamètres SVM (GridSearchCV)
On cherche de bons hyperparamètres `C` et `gamma` pour un noyau RBF.

**À faire :**
- compléter la grille `param_grid`
- lancer `GridSearchCV`
- évaluer le meilleur modèle sur le test


In [ ]:
param_grid = {
    "svc__C": [...],       # TODO: ex [0.1, 1, 10, 100]
    "svc__gamma": [...]    # TODO: ex [0.001, 0.01, 0.1, "scale"]
}

svm = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(kernel="rbf"))
])

try:
    gs = GridSearchCV(
        svm,
        param_grid=param_grid,
        cv=5,
        scoring="accuracy",
        n_jobs=-1
    )
    gs.fit(X_train, y_train)

    print("Meilleurs hyperparamètres:", gs.best_params_)
    print("Meilleur score CV:", gs.best_score_)

    best_model = gs.best_estimator_
    y_pred = best_model.predict(X_test)

    print("Accuracy test (best):", accuracy_score(y_test, y_pred))

    cm = confusion_matrix(y_test, y_pred)
    ConfusionMatrixDisplay(cm).plot()
    plt.title("Matrice de confusion — SVM (meilleur modèle)")
    plt.show()
except Exception as e:
    print(" Complétez la grille param_grid (TODO) pour lancer GridSearchCV:", e)


### 11) Clustering hiérarchique (Agglomerative Clustering)
On revient sur les données 2D `make_blobs` pour visualiser.

**Hyperparamètres importants :**
- `n_clusters`
- `linkage` : 'ward', 'complete', 'average', 'single'
- `metric` (selon version) : distance utilisée

**À faire :**
- essayer plusieurs linkages et comparer visuellement


In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

# On standardise pour que les distances soient comparables (souvent utile)
Xc = StandardScaler().fit_transform(X)

configs = [
    # TODO: testez différents linkages
    {"n_clusters": 4, "linkage": "ward"},
    {"n_clusters": 4, "linkage": "complete"},
    {"n_clusters": 4, "linkage": "average"},
    {"n_clusters": 4, "linkage": "single"},
]

for cfg in configs:
    model = AgglomerativeClustering(**cfg)
    labels = model.fit_predict(Xc)

    sil = silhouette_score(Xc, labels)
    plt.figure()
    plt.scatter(Xc[:, 0], Xc[:, 1], c=labels, s=15)
    plt.title(f"Agglomératif — {cfg} | silhouette={sil:.3f}")
    plt.show()


### 12) Comparer (optionnel) : métriques de clustering si labels disponibles
Sur `make_blobs`, on a `y_true` (labels générés). Ce n'est pas 'réel' en non supervisé, mais ça permet d'évaluer.

**À faire :** calculer ARI (Adjusted Rand Index) pour comparer k-means sklearn vs hiérarchique.


In [ ]:
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans

Xc = StandardScaler().fit_transform(X)

km = KMeans(n_clusters=4, n_init=20, random_state=42)
labels_km = km.fit_predict(Xc)

agg = AgglomerativeClustering(n_clusters=4, linkage="ward")
labels_agg = agg.fit_predict(Xc)

# TODO: calculez ARI pour les deux
ari_km = ...   # TODO
ari_agg = ...  # TODO

print("ARI k-means:", ari_km)
print("ARI agglomératif:", ari_agg)


---
## Questions / À rendre
1. Dans votre k-means from scratch :
   - quelle loss avez-vous implémentée ?
   - quel critère d'arrêt utilisez-vous ?
   - que se passe-t-il si un cluster se vide ?
2. SVM : comment `C` et `gamma` influencent-ils la frontière de décision ? (intuition)
3. Comparez la matrice de confusion avant/après GridSearchCV : quels chiffres sont les plus confondus ?
4. Clustering hiérarchique : quel linkage donne la meilleure silhouette sur ce dataset ? Pourquoi ?
